In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
<!DOCTYPE html>
<html>
<head>
    <title>Reconhecimento de Imagens com Webcam</title>
    <style>
        body { font-family: Arial, sans-serif; margin: 20px; }
        #camera { width: 640px; height: 480px; border: 1px solid #ccc; }
        #result { margin-top: 20px; font-size: 18px; font-weight: bold; }
        button { padding: 10px 15px; font-size: 16px; cursor: pointer; }
    </style>
</head>
<body>
    <h1>Reconhecimento de Objetos com Webcam</h1>
    <video id="camera" autoplay></video>
    <div>
        <button id="capture">Capturar e Analisar</button>
        <button id="startContinuous">Iniciar Análise Contínua</button>
        <button id="stopContinuous" disabled>Parar Análise</button>
    </div>
    <div id="result">Aguardando análise...</div>

    <script>
        const video = document.getElementById('camera');
        const captureBtn = document.getElementById('capture');
        const startBtn = document.getElementById('startContinuous');
        const stopBtn = document.getElementById('stopContinuous');
        const resultDiv = document.getElementById('result');

        let continuousMode = false;
        let intervalId = null;

        // Acessar a webcam
        navigator.mediaDevices.getUserMedia({ video: true })
            .then(stream => {
                video.srcObject = stream;
            })
            .catch(err => {
                console.error("Erro ao acessar a webcam:", err);
                resultDiv.textContent = "Erro ao acessar a webcam: " + err.message;
            });

        // Função para capturar e enviar imagem
        function captureAndAnalyze() {
            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0, canvas.width, canvas.height);

            canvas.toBlob(blob => {
                const formData = new FormData();
                formData.append('image', blob, 'capture.jpg');

                fetch('/analyze', {
                    method: 'POST',
                    body: formData
                })
                .then(response => response.json())
                .then(data => {
                    resultDiv.textContent = `Resultado: ${data.class} (${Math.round(data.confidence * 100)}%)`;
                })
                .catch(error => {
                    console.error('Erro:', error);
                    resultDiv.textContent = 'Erro ao analisar a imagem';
                });
            }, 'image/jpeg', 0.8);
        }

        // Event listeners
        captureBtn.addEventListener('click', captureAndAnalyze);

        startBtn.addEventListener('click', () => {
            continuousMode = true;
            startBtn.disabled = true;
            stopBtn.disabled = false;
            intervalId = setInterval(captureAndAnalyze, 2000); // Analisar a cada 2 segundos
        });

        stopBtn.addEventListener('click', () => {
            continuousMode = false;
            startBtn.disabled = false;
            stopBtn.disabled = true;
            clearInterval(intervalId);
        });
    </script>
</body>
</html>